# Ejercicio 7: Bases de Datos Vectoriales

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

In [ ]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

In [ ]:
# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

In [ ]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

### Guardar los Embeddings

In [ ]:
import numpy as np

embeddings_file_path = "wikipedia_embeddings_e5.npy"
np.save(embeddings_file_path, embeddings)
print(f"Embeddings guardados en: {embeddings_file_path}")

### Cargar los Embeddings

In [ ]:
import numpy as np

embeddings_file_path = "wikipedia_embeddings_e5.npy"

# Carga el array de embeddings
embeddings = np.load(embeddings_file_path)

In [ ]:
print(embeddings.shape, embeddings.dtype)

In [ ]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_embedding = embed_query(query_text)
query_embedding.shape

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [ ]:
!pip install faiss-cpu

In [ ]:
# código base para FAISS
import faiss
import numpy as np

# Asumiendo `embeddings` en un array NxD
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

D, I = index.search(query_embedding, k=10)

In [ ]:
print(chunks_df["text"].iloc[I[0].tolist()])

## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?


In [ ]:
!pip install -U qdrant-client

In [ ]:
from qdrant_client import models, QdrantClient

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="wikipedia",
    vectors_config=models.VectorParams(
        size=model.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

In [ ]:
client.upload_points(
    collection_name="wikipedia",
    points=[
        models.PointStruct(
            id=idx,
            vector=embeddings[idx].tolist(), # Asigna el embedding correcto para este punto
            payload=row.to_dict()           # Convierte la fila del DataFrame a un diccionario para el payload
        )
        for idx, row in chunks_df.iterrows() # Itera sobre las filas del DataFrame
    ],
)

In [ ]:
hits = client.query_points(
    collection_name="wikipedia",
    query=query_embedding[0].tolist(),
    limit=10,
).points

for hit in hits:
    print(hit.payload, "score:", hit.score)

Definición de la función `qdrant_search(query_embedding, k)`

In [ ]:
def qdrant_search(query_embedding, k):
  hits = client.query_points(
    collection_name="wikipedia",
    query=query_embedding[0].tolist(),
    limit=k,
  ).points
  return hits

In [ ]:
query_text = "multidimensional space"
query_embedding = embed_query(query_text)
hits = qdrant_search(query_embedding, k=5)
for hit in hits:
    print(hit.payload, "score:", hit.score)

### Liberación de recursos de RAM

In [ ]:
import gc

# Eliminar el DataFrame original
if 'df' in locals():
    del df

# Eliminar la lista de pasajes
if 'passages' in locals():
    del passages

# Forzar el recolector de basura de Python
gc.collect()

Eliminar la colección de Qdrant

In [ ]:
collection_name = "wikipedia"

# Verifica si la colección existe antes de intentar eliminarla
if client.collection_exists(collection_name):
    client.delete_collection(collection_name=collection_name)

## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


In [ ]:
!pip install -U pymilvus

In [ ]:
!pip install pymilvus[milvus_lite]

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

In [ ]:
COLLECTION_NAME = "wikipedia"

client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=embeddings.shape[1],
    primary_field_name="id",
    vector_field_name="embedding",
    auto_id=False,
)

In [ ]:
milvus_data = []

for i, row in chunks_df.iterrows():
    milvus_data.append({
        "id": int(i),
        "embedding": embeddings[i].tolist(),
        "text": row["text"],
        "doc_id": int(row["doc_id"]),
        "chunk_id": int(row["chunk_id"]),
    })

In [ ]:
batch_size = 5000

for i in range(0, len(milvus_data), batch_size):
    batch = milvus_data[i:i + batch_size]
    client.insert(collection_name=COLLECTION_NAME, data=batch)
    print(f"Insertados {len(batch)} puntos. Total insertados: {i + len(batch)}")

In [ ]:
def milvus_search(query_embedding, k):
  res = client.search(
      collection_name=COLLECTION_NAME,  # target collection
      data=query_embedding,  # query vectors: debe ser una lista de vectores
      limit=k,  # number of returned entities
      output_fields=["doc_id", "chunk_id", "text"],  # specifies fields to be returned
  )
  return res

In [ ]:
query_text = "Battery measuring"
query_embedding = embed_query(query_text)

res = milvus_search(query_embedding.tolist(), k=10)
for hit in res:
  for hitt in hit:
    print(hitt)

## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- ¿Cómo describirías el trade-off de complejidad vs expresividad?


In [ ]:
!pip install -U weaviate-client[agents]

In [ ]:
from weaviate.client import WeaviateClient
from weaviate.embedded import EmbeddedOptions

client = WeaviateClient(
    embedded_options=EmbeddedOptions()
)

In [ ]:
from weaviate.client import WeaviateClient
import weaviate.classes as wvc

client.collections.create(
    name="WikipediaChunk",
    properties=[
        wvc.config.Property(name="text", data_type=wvc.config.DataType.TEXT),
        wvc.config.Property(name="doc_id", data_type=wvc.config.DataType.INT),
        wvc.config.Property(name="chunk_id", data_type=wvc.config.DataType.INT)
    ],
    vector_config=wvc.config.Configure.Vectorizer.none(),
    vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
        distance_metric=wvc.config.VectorDistances.COSINE
    )
)

## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
- ¿Qué limitaciones ves para un sistema en producción?


In [ ]:
!pip install -U chromadb

In [ ]:
import chromadb

client = chromadb.Client()
print("ChromaDB client initialized in-memory.")

In [ ]:
collection_name = "wikipedia_chunks_chroma"

# Get or create the collection to avoid 'already exists' error
collection = client.get_or_create_collection(name=collection_name)
print(f"Collection '{collection_name}' is ready.")

# Prepare data for insertion
ids = [str(i) for i in chunks_df.index.tolist()]
documents = chunks_df["text"].tolist()

metadatas = []
for i, row in chunks_df.iterrows():
    metadatas.append({"doc_id": int(row["doc_id"]), "chunk_id": int(row["chunk_id"])}) # Ensure int type for metadata

batch_size = 5000 # Using a batch size smaller than the reported max batch size

for i in range(0, len(ids), batch_size):
    batch_ids = ids[i:i + batch_size]
    batch_embeddings = embeddings[i:i + batch_size].tolist()
    batch_documents = documents[i:i + batch_size]
    batch_metadatas = metadatas[i:i + batch_size]

    collection.add(
        embeddings=batch_embeddings,
        documents=batch_documents,
        metadatas=batch_metadatas,
        ids=batch_ids
    )
    print(f"Inserted {len(batch_ids)} documents. Total inserted: {i + len(batch_ids)}")

print(f"Finished inserting {len(ids)} documents into '{collection_name}' collection.")

In [ ]:
def chroma_search(query_embedding, k):
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=k,
        include=['documents', 'distances', 'metadatas']
    )
    return results

In [ ]:
query_text = "Battery measuring"
query_embedding = embed_query(query_text)

hits = chroma_search(query_embedding, k=5)

# Print the results in a readable format
print("ChromaDB Search Results (k=5) for query: 'Battery measuring'")
for i in range(len(hits['ids'][0])):
    print(f"---\nID: {hits['ids'][0][i]}\nScore (distance): {hits['distances'][0][i]}\nText: {hits['documents'][0][i]}\nMetadata: {hits['metadatas'][0][i]}")

## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?


In [ ]:
!pip install psycopg2-binary

In [ ]:
import psycopg2

DB_NAME = "mydatabase"
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"

conn = None
cursor = None

try:
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    cursor = conn.cursor()

    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    conn.commit()

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
